<a id="plan"></a>

<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  Dataset Lilly (pkg)
  </div> 
  
<div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 30px; 
      text-align: center; 
      padding: 15px; 
      margin: 10px;">
  VII. Modèle linguistique
  </div> 

  <div style="font-variant: small-caps; 
      font-weight: normal; 
      font-size: 20px; 
      text-align: center; 
      padding: 15px;">
  ALTRAN - Projet MODINS
  </div> 

  <div style=" float:right; 
      font-size: 12px; 
      line-height: 12px; 
  padding: 10px 15px 8px;">
  Benoît COURBON  |
  Jean-baptiste AUJOGUE
  </div> 

  <div style=" display: inline-block;
      font-family: 'Lato',
      sans-serif;
      font-size: 12px;
      font-weight: bold;
      line-height: 12px;
      letter-spacing: 1px;
      padding: 10px 15px 8px;">
  29/04/2019
  </div>

In [1]:
# path to anaconda : C:\ProgramData\Anaconda3

from __future__ import unicode_literals, print_function, division
import os
from io import open
import unicodedata
import string
import time
import math
import re
import json
import random
import itertools
import copy

In [2]:
import sys
sys.version

'3.7.0 (default, Jun 28 2018, 08:04:48) [MSC v.1912 64 bit (AMD64)]'

In [3]:
# for manipulating dataframes, arrays, containers...
import numpy as np
#np.set_printoptions(threshold=np.nan)
import pandas as pd
import unidecode
from unidecode import unidecode

# equivalent of numpy for very large arrays (such as a database of pretrained word vectors)
# optimized for SQL-like operations, not for math computations
# see https://bcolz.readthedocs.io/en/latest/intro.html

# installer avec la commande : conda install bcolz
import bcolz

import pickle

In [4]:
# for deep learning
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

print(torch.__version__)

use_cuda = False
print(torch.cuda.is_available())

import warnings
warnings.filterwarnings("ignore")

0.4.1
True


In [5]:
# for NLP
import gensim
from gensim.models import KeyedVectors

import nltk
#nltk.download()
from nltk.tokenize import sent_tokenize, word_tokenize, RegexpTokenizer
from nltk.stem.porter import PorterStemmer

#from textblob import TextBlob

In [6]:
import warnings
warnings.filterwarnings('ignore')

USE_CUDA = torch.cuda.is_available()
print(USE_CUDA)
device = torch.device("cuda" if USE_CUDA else "cpu")

True


In [7]:
np.random.seed(42)

path_to_rep = 'C:\\Users\\jbaujogue\\Desktop\\MODINS\\use case Lilly'
path_to_NLP = 'C:\\Users\\jbaujogue\\Desktop\\NLP'

<a id="corpus"></a>

# 0 Préparation

<a id="import"></a>

### 0.1 Import du corpus

[Retour à la table des matières](#plan)

Import du corpus de tickets Lilly, avec chaque ticket sous la forme d'une liste nettoyée de tokens

In [8]:
df_Lilly = pd.read_excel(path_to_rep + '\data\Tickets_Lilly_pkg\\tickets_Lilly_05_03_2019.xlsx' )
df_Lilly = df_Lilly.reset_index(drop=True)
print(df_Lilly.shape)
df_Lilly.head()

(3743, 8)


,Number,Created,Problem,Resolution Code,Short Description,Description,Resolve Notes,Analyse
0,INC2976351,2016-06-01 06:54:32,NaN,People/System use,AST : PMX-PKG : Démarrage étape de production ...,Impossible de démarrer l'étape de production s...,• [Résumé] Impossible de démarrer l'étape de p...,Hscope/Duplicate/Cslt/Action
1,INC2976936,2016-06-01 09:56:53,NaN,NaN,PMX-PKG : Alarme type 2 non vue par l'équipement,Une alarme s'est déclenchée sur le limos LPF-E...,• [Résumé]\n\n• [Catégorie] \n\n• [Résolution]...,Matrikon/Alarme
2,INC2977191,2016-06-01 10:51:31,NaN,Monitoring,PMX-PKG : ticket journalier MO3524 du 01/06/2016,01/06/2016 00:00:00 => 01/06/2016 10:47:14\n-...,• [Catégorie] Monitoring des erreurs d'interfa...,Dailyinterfacemonitoring
3,INC2981590,2016-06-02 09:55:10,NaN,NaN,PMX-PKG : ref_relation = WESSTOCKREMOVAL (LC ...,Message envoyé le 01/06/2016 12:57:07\n\nid = ...,• [Résumé] Message d'erreur reçu sur l'interfa...,Lcmovementerror
4,INC2981597,2016-06-02 09:56:36,NaN,NaN,PMX-PKG : ref_relation = WESSTOCKREMOVAL (LC =...,Message envoyé le 01/06/2016 12:59:07\n\nid = ...,• [Résumé] Message d'erreur reçu sur l'interfa...,Lcmovementerror


In [9]:
def fusion(liste1, liste2):
    return [a.lower() + ' ' + b.lower() for a,b in zip(liste2, liste1)]

In [18]:
titres = ['Short Description : ', 'Description : ', 'Resolve Notes : ']
X_raw = [' . '.join(fusion([str(el) for el in els], titres )) \
            for els in zip(df_Lilly['Short Description'].values.tolist(),
                           df_Lilly['Description'].values.tolist(),
                           df_Lilly['Resolve Notes'].values.tolist()
                          )]
#X_raw = [[[word.strip() for word in sentence.split()] + ['.'] for sentence in teX_rawt.split('.')] for teX_rawt in X_raw]
# nettoyage préalable
X_raw = [re.sub('\n', ' . ', teX_rawt) for teX_rawt in X_raw] # on remplace chaque retour chariot par un point espacé
X_raw = [re.sub('arretmajeur', 'arret majeur', teX_rawt) for teX_rawt in X_raw] 
X_raw = [re.sub('=>', ' => ', teX_rawt) for teX_rawt in X_raw] 
X_raw = [re.sub('->', ' -> ', teX_rawt) for teX_rawt in X_raw] 
X_raw = [re.sub('(?P<val>\S)\.', '\g<val> .', teX_rawt) for teX_rawt in X_raw] # on insere un espace à la fin de chaque hashtag
X_raw = [re.sub('#(?P<val>[0-9]+)(?P<val2>[^0-9])', '#\g<val> \g<val2>', teX_rawt) for teX_rawt in X_raw] # on insere un espace à la fin de chaque hashtag
X_raw = [teX_rawt + '.' for teX_rawt in X_raw] # on insere un point a la fin de chaque teX_rawte

In [15]:
def CleanAndComputeAlignement(X, dict_sub, dict_replace) :
    def sub2replace(dict_sub, corpus):
        '''corpus is [str]'''
        dict_replace = {}
        for key in dict_sub.keys():
            occ_list = []
            for text in corpus : occ_list += re.findall(key, text)
            for word in occ_list :
                dict_replace[word] = dict_sub[key]
        return dict_replace  



    def augmentValeur(dict_replace):
        for key, value in dict_replace.items() :
            if len(key) > len(value) : dict_replace[key] = value + ''.join([' ']*(len(key) - len(value)))
        return dict_replace



    def applyReplace(dict_replace, corpus):
        new_corpus = []
        caracters_alignement_list = []
        for text in corpus :
            # initialisation du dictionnaire d'alignement de caractères
            carac_dict = {}
            for i in range(len(text)): carac_dict[i] = i

            # mise à jour du dictionnaire d'alignement pour chaque mot
            for word in dict_replace.keys():
                diff = len(dict_replace[word]) - len(word)
                if diff > 0 :
                    start = text.find(word)
                    while start != -1 :
                        reste = start + len(word)
                        #if '#' in word : print(word, text[start : reste], diff)
                        for i in range(reste, len(text)) : carac_dict[i] += diff
                        start = text.find(word, reste)
            caracters_alignement_list.append(carac_dict)

            # remplacement de chaque mot
            for word in dict_replace.keys(): text = text.replace(word, dict_replace[word])
            new_corpus.append(text)

        return new_corpus, caracters_alignement_list


    def removeSpecialCaracters(text):
        def replace(c):
            return c if unicodedata.category(c) != 'Mn' else ''
        def unicodeToAscii(s):
            """Turn a Unicode string to plain ASCII, thanks to http://stackoverflow.com/a/518232/2809427"""
            return ''.join(replace(c) for c in unicodedata.normalize('NFD', s))
        text = unicodeToAscii(text)
        text = re.sub(r"[^a-zA-Z\.]", r" ", text)
        text = text.lower()
        return text



    def cleanPoints(corpus):
        new_corpus = []
        for text in corpus :
            new_text = ''
            vu = 0
            while vu < len(text) and (text[vu] == '.' or text[vu] == ' ') :
                new_text += ' '
                vu += 1
            while vu < len(text) :
                if text[vu] == '.' :
                    new_text += '.'
                    vu += 1
                    while vu < len(text) and (text[vu] == '.' or text[vu] == ' ') :
                        new_text += ' '
                        vu += 1
                else :
                    new_text += text[vu]
                    vu += 1

            # on place correctement le point de la fin
            #new_text = new_text[:-1] + '.'
            #vu = len(new_text)-2
            #while vu >=0 and (new_text[vu] == '.' or new_text[vu] == ' ') :
            #    new_text = new_text[:vu] + ' ' + new_text[vu + 1:]
            #    vu -= 1
            new_corpus.append(new_text)
        return new_corpus



    def countPadding(corpus):
        def computePadding(text, word, vu):
            padding = 0
            while (vu + len(word) + padding) < len(text) and text[vu + len(word) + padding] == ' ' :
                padding += 1
            return padding

        padding_list = []
        for text in corpus :
            words = [word for sentence in text.split('.') for word in sentence.split() + ['.'] if len(sentence.split())>0] #text.split()
            paddings = []
            vu = 0
            while text[vu] == ' ':
                vu += 1
            paddings.append(['SOS', vu])
            for word in words :
                padding = computePadding(text, word, vu)
                paddings.append([word, padding])
                vu += len(word) + padding
            padding_list.append(paddings)
        return padding_list



    def car2Words(padding_list) :
        caracter2word_dict = []
        for paddings in padding_list :
            text = ''.join(el[0] + ''.join([' ']*el[1]) for el in paddings)[3:]
            carac2word = {}
            vu = 0
            for n, el in enumerate(paddings) :
                length = len(el[0]) + el[1] if n > 0 else el[1]
                for i in range(vu, vu + length) :
                    carac2word[i] = el[0] if text[i] != ' ' else 0
                vu += length
            caracter2word_dict.append(carac2word)
        return caracter2word_dict



    def word2Position(caracter2word_list):
        new_list = []
        for car2word in caracter2word_list :
            new_car2word = {}
            c  = 0
            word_count = 0
            line_count = 0
            while c <= max(list(car2word.keys())) :
                w = car2word[c]
                if w == '.' :
                    new_car2word[c] = [line_count, word_count]
                    line_count += 1
                    word_count = 0
                    c += 1
                elif w != 0 :
                    while c <= max(list(car2word.keys())) and car2word[c] == w :
                        new_car2word[c] = [line_count, word_count]
                        c += 1
                    word_count += 1
                else :
                    new_car2word[c] = car2word[c]
                    c+= 1
            new_list.append(new_car2word)
        return new_list  




    def rawCar2Position(rawcar2car_list, car2pos_list):
        new_list = []
        for rawcar2car, car2pos in zip(rawcar2car_list, car2pos_list):
            rawcar2pos = {}
            for key, value in rawcar2car.items():
                rawcar2pos[key] = car2pos[value]
            new_list.append(rawcar2pos)
        return new_list

    
    
    dict_replace_add = sub2replace(dict_sub, X)
    dict_replace.update(dict_replace_add)
    #print('Il y a au total {} opération .replace à effectuer sur le corpus'.format(len(list(dict_replace.keys()))))
    dict_replace = augmentValeur(dict_replace)
    
    X2, rawcar2car_list = applyReplace(dict_replace, X)
    X3 = [removeSpecialCaracters(text) for text in X2]
    X4 = cleanPoints(X3)
    
    padding_list = countPadding(X4)
    car2word_list = car2Words(padding_list)
    car2pos_list = word2Position(car2word_list)
    rawcar2pos_list = rawCar2Position(rawcar2car_list, car2pos_list)
    
    X_final = [[sentence.split() + ['.'] for sentence in text.split('.') if len(sentence.split())>0] for text in X4]
    return X_final, rawcar2pos_list

In [16]:
dict_sub = {'[^=-]>\s*0' : 'positive',
            '#[0-9]+ ' : 'hashtag ',
            #'[0-9]+\:[0-9]+\:[0-9]*' : 'heurenumber',
            #'[0-9]+\-[0-9]+\-[0-9]+' : 'datenumber',
            #'[0-9]+\/[0-9]+\/[0-9]+' : 'datenumber',
            #'[0-9]+\.[0-9]+\.[0-9]+' : 'datenumber',
            '•' : '.'}

dict_replace = {'=>' : 'donc',
                '->' : 'donc',
                '.zip'  : ' zip',
                'loc.'  : 'loc ',
                'checkliste' : 'checklist',
                'close loop' : 'closeloop',
                'SUPER.' : 'SUPER ',
                'imprimente' : 'imprimante',
                'dim.' : 'dimanche'
                }

In [19]:
X, rawcar2pos_list = CleanAndComputeAlignement(X_raw, dict_sub, dict_replace)

In [21]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


X_flat = [flatten(text) for text in X]

Formation du language

In [23]:
class Lang:
    def __init__(self, name, init = 3):
        self.name = name
        if init == 0 :
            self.word2index = {}
            self.word2count = {}
            self.index2word = {}
            self.n_words = 0
        elif init == 3 :
            self.word2index = {"SOS": 0, "EOS": 1, "UNK": 2}
            self.word2count = {"SOS": 0, "EOS": 0, "UNK": 0}
            self.index2word = {0: "SOS", 1: "EOS", 2: "UNK"}
            self.n_words = 3

        
    def addWord(self, word):
        '''Add a word to the language'''
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1
            
            
    def addSentence(self, sentence):
        '''Add to the language all words of a sentence'''
        if type(sentence) == str :
            for word in nltk.word_tokenize(sentence): # sentence.split()
                self.addWord(word)
        elif type(sentence) == list :
            for word in sentence:
                self.addWord(word)            
            
            
    def addDescriptions(self, descriptions, lvl = 1):
        '''Add to the language all words contained into : either all user utterances 
          (if i = 0) or all bot utterances (if i = 1), of a list of dialogues'''
        for description in descriptions :
            
            # si la description est une seule ligne
            if type(description) == str :
                    try :
                        if lvl == 2 :
                            for sentence in nltk.sent_tokenize(description) :
                                self.addSentence(sentence)
                        else :
                            self.addSentence(description)
                    except IndexError:
                        print("Problem with {}".format(description))
                        
            # si la description est une liste de lignes
            elif type(description) == list :
                for line in description:
                    try :
                        self.addSentence(line)
                    except IndexError:
                        print("Problem with {}".format(line))
                              
            # sinon
            else :
                print("Problem with {}".format(description))

In [24]:
def generateLanguageFromWordList(liste, init = 0):
    lang = Lang('descriptions', init)
    lang.addSentence(liste)
    print("Mots comptés : ", lang.n_words)
    return lang


def generateLanguageFromNumpy(descriptions, init = 0, lvl = 1):
    lang = Lang('descriptions', init)
    lang.addDescriptions(descriptions)
    print("Mots comptés : ", lang.n_words)
    return lang


def generateLanguageFromPanda(df, i):
    descriptions = df.ix[:, i].values
    lang = Lang('descriptions')
    lang.addDescriptions(descriptions)
    print("Mots comptés : ", lang.n_words)
    return lang

In [25]:
lang = generateLanguageFromNumpy(X_flat, init = 3, lvl = 1)

Mots comptés :  7238


<a id="embedding"></a>

### 0.2 Word Embedding

[Retour à la table des matières](#plan)

#### 0.2.1 Modèle Skip-gram de Gensim

In [66]:
from gensim.models import Word2Vec
import multiprocessing

In [67]:
data_gensim_skipgram = flatten(X) # ------ [[str]] : al list of list of tockens

In [68]:
skipgram_gensim = Word2Vec(data_gensim_skipgram, 
                           size = 100, 
                           window = 5, 
                           min_count = 1, 
                           negative = 20, 
                           iter = 500,
                           sg = 1,
                           workers = multiprocessing.cpu_count())

Obtenir la table des vecteurs

In [70]:
T = skipgram_gensim.wv.vectors
print('T est de type {} et de taille {}'.format(type(T), T.shape))

T est de type <class 'numpy.ndarray'> et de taille (6488, 100)


#### 0.2.2 Modèle Skip-gram sur mesure


In [54]:
def flatten(description) :
    '''Baisse le nombre de niveaux de 1 dans la description'''
    flatten = []
    for line in description :
        flatten += line
    return flatten


def generateNgrams(descriptions, context_size = 5) :
    '''descriptions = [[str]]'''
    data = []
    for description in descriptions :
        line = flatten(description)
        line = ['SOS' for i in range(context_size)] + line + ['EOS' for i in range(context_size)] 
        for i in range(context_size, len(line) - context_size):
            context = line[i-context_size : i] + line[i+1 : i+context_size+1]
            target = [line[i]]
            data.append((context, target))
    return data

In [26]:
class SkipGram(nn.Module):
    def __init__(self, device, 
                 lang, 
                 context_size, 
                 embedding_dim, 
                 hidden_dim):
        
        super(SkipGram, self).__init__()
        self.device = device
        self.lang = lang
        self.output_lang = copy.copy(lang)
        self.output_lang.addDescriptions(['SOS', 'UNK', 'EOS'])
        self.context_size = 2 * context_size
        self.embedding = nn.Embedding(lang.n_words, embedding_dim)
        self.linear_1 = nn.Linear(embedding_dim, 2*context_size * hidden_dim)
        self.linear_2 = nn.Linear(hidden_dim, self.output_lang.n_words)
        
        
    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(1, -1)).to(self.device)
        return result
    

    def forward(self, word):
        input = self.variableFromSentence(word)                   # size = (1, 1)
        embed = self.embedding(input)#.view((1, -1))              # size = (1, embedding_dim)
        out = self.linear_1(embed).view((self.context_size, -1))  # size = (2 * context_size , hidden_dim)
        out = F.relu(out)                                         # size = (2 * context_size , hidden_dim)
        out = self.linear_2(out)                                  # size = (2 * context_size , lang.n_words)
        log_probs = F.log_softmax(out)                            # size = (2 * context_size , lang.n_words)
        return log_probs

In [27]:
class SkipGramTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 print_every=100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.print_every = print_every
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def trainLoop(self, agent, context, word, weights, optimizer, learning_rate):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        
        log_probs = agent(word)
        context_var = agent.variableFromSentence(context).view(-1)
        loss = self.criterion(log_probs, context_var)
        loss = torch.sum(weights * loss)
        loss_diff = 0
        for i in range(agent.context_size) :
            topv, topi = log_probs[i].data.topk(1)
            ni = topi[0][0]
            loss_diff = loss_diff + 1 if ni != context_var[i].data[0] else loss_diff
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff
        
        
    def train(self,
              agent, 
              ngrams, 
              weights,
              n_iters = 100,
              n_epochs = None,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        weights = Variable(torch.FloatTensor(weights)).to(self.device)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        if n_epochs is None :
            for iter in range(1, n_iters + 1):
                couple = random.choice(ngrams)
                context = couple[0] 
                target = couple[1] 
                loss, loss_diff_mots = self.trainLoop(agent, context, target, weights, optimizer, learning_rate)
                # quantité d'erreurs sur la réponse i
                print_loss_total += loss
                print_loss_diff_mots_total += loss_diff_mots       
                if iter % (self.print_every) == 0:
                    print_loss_avg = print_loss_total / self.print_every
                    print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                    print_loss_total = 0
                    print_loss_diff_mots_total = 0
                    print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                                 iter, iter / n_iters * 100, 
                                                      print_loss_avg, print_loss_diff_mots_avg))
        else :
            for epoch in range(n_epochs):
                print('epoch ' + str(epoch))
                np.random.shuffle(ngrams)
                for couple in ngrams :
                    context = couple[0] 
                    target = couple[1] 
                    loss, loss_diff_mots = self.trainLoop(agent, context, target, weights, optimizer, learning_rate)

                    # quantité d'erreurs sur la réponse i
                    print_loss_total += loss
                    print_loss_diff_mots_total += loss_diff_mots       

                    if iter % (self.print_every) == 0:
                        print_loss_avg = print_loss_total / self.print_every
                        print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                        print_loss_total = 0
                        print_loss_diff_mots_total = 0
                        print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                                     iter, iter / n_iters * 100, 
                                                          print_loss_avg, print_loss_diff_mots_avg))

In [28]:
def weightsList(n):
    half = (n-1)/2
    weights = [n**2 -(half-i)**2 for i in range(n)]
    weights = [el/sum(weights) for el in weights]
    return weights

Entrainement du modèle

In [64]:
skipgram = SkipGram(device, lang, context_size = 5, embedding_dim = 75, hidden_dim = 75)
skipgram = skipgram.to(device)
skipgram_trainer = SkipGramTrainer(device = device)

In [65]:
Ngrams = generateNgrams(X, context_size = 5)

In [66]:
skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 40000, learning_rate=0.01)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.005)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.0025)
#skipgram_trainer.train(skipgram, Ngrams, weights = weightsList(10), n_iters = 30000, learning_rate=0.001)

0m 1s (- 8m 21s) (100 0%) 8.6615 9.31
0m 2s (- 8m 14s) (200 0%) 8.3966 7.81
0m 3s (- 8m 14s) (300 0%) 7.9387 7.37
0m 4s (- 8m 11s) (400 1%) 7.6289 7.46
0m 6s (- 8m 8s) (500 1%) 7.3045 7.53
0m 7s (- 8m 5s) (600 1%) 7.2159 7.55
0m 8s (- 8m 5s) (700 1%) 6.9773 7.34
0m 9s (- 8m 4s) (800 2%) 7.3433 7.89
0m 11s (- 8m 3s) (900 2%) 6.9874 7.49
0m 12s (- 8m 0s) (1000 2%) 7.0193 7.57
0m 13s (- 7m 59s) (1100 2%) 6.9350 7.46
0m 14s (- 7m 58s) (1200 3%) 7.2537 7.85
0m 16s (- 7m 56s) (1300 3%) 6.8125 7.40
0m 17s (- 7m 55s) (1400 3%) 6.9897 7.67
0m 18s (- 7m 54s) (1500 3%) 6.8655 7.45
0m 19s (- 7m 54s) (1600 4%) 6.7800 7.33
0m 21s (- 7m 53s) (1700 4%) 6.8878 7.56
0m 22s (- 7m 51s) (1800 4%) 6.7829 7.45
0m 23s (- 7m 51s) (1900 4%) 6.7094 7.43
0m 24s (- 7m 49s) (2000 5%) 6.9370 7.81
0m 25s (- 7m 48s) (2100 5%) 6.8034 7.65
0m 27s (- 7m 46s) (2200 5%) 6.8269 7.61
0m 28s (- 7m 45s) (2300 5%) 6.7266 7.65
0m 29s (- 7m 44s) (2400 6%) 6.6305 7.67
0m 30s (- 7m 44s) (2500 6%) 6.6411 7.53
0m 32s (- 7m 44s) (2600

4m 13s (- 4m 8s) (20200 50%) 4.6532 6.74
4m 14s (- 4m 7s) (20300 50%) 4.5873 6.68
4m 15s (- 4m 5s) (20400 51%) 4.5567 6.74
4m 17s (- 4m 4s) (20500 51%) 4.6774 6.89
4m 18s (- 4m 3s) (20600 51%) 4.9188 7.06
4m 19s (- 4m 2s) (20700 51%) 4.5559 6.77
4m 20s (- 4m 0s) (20800 52%) 4.2201 6.46
4m 22s (- 3m 59s) (20900 52%) 4.3170 6.56
4m 23s (- 3m 58s) (21000 52%) 4.6413 6.99
4m 24s (- 3m 56s) (21100 52%) 4.5372 6.83
4m 25s (- 3m 55s) (21200 53%) 4.3307 6.49
4m 27s (- 3m 54s) (21300 53%) 4.6840 6.79
4m 28s (- 3m 53s) (21400 53%) 4.6583 6.91
4m 29s (- 3m 51s) (21500 53%) 4.3774 6.50
4m 30s (- 3m 50s) (21600 54%) 4.4222 6.63
4m 31s (- 3m 49s) (21700 54%) 4.6390 6.85
4m 33s (- 3m 48s) (21800 54%) 4.5037 6.67
4m 34s (- 3m 46s) (21900 54%) 4.6484 6.85
4m 35s (- 3m 45s) (22000 55%) 4.7608 7.11
4m 36s (- 3m 44s) (22100 55%) 4.3271 6.68
4m 38s (- 3m 43s) (22200 55%) 4.2208 6.14
4m 39s (- 3m 41s) (22300 55%) 4.3677 6.53
4m 40s (- 3m 40s) (22400 56%) 4.4364 6.70
4m 41s (- 3m 39s) (22500 56%) 4.3389 6.66

8m 18s (- 0m 1s) (39900 99%) 3.9669 6.40
8m 19s (- 0m 0s) (40000 100%) 4.2651 6.35


In [67]:
# sauver
#torch.save(skipgram.state_dict(), path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_skipgram.pth')

# charger
#skipgram.load_state_dict(torch.load(path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_skipgram.pth'))

Obtenir la table des vecteurs

In [68]:
T = skipgram.embedding.weight.cpu().detach().numpy()
print('T est de type {} et de taille {}'.format(type(T), T.shape))

T est de type <class 'numpy.ndarray'> et de taille (6474, 75)


L'alignement mot-indice est donné par le dictionnaire lang.word2index du language chargé dans le modèle

#### 0.2.3 Modèle Skip-gram de FastText

In [26]:
from gensim.models.fasttext import FastText as FT_gensim
from gensim.test.utils import datapath

In [27]:
fastText_gensim = FT_gensim(size = 100, 
                           window = 5, 
                           min_count = 1, 
                           negative = 10,
                           sg = 1)

In [28]:
X_fasttext = [['SOS'] + text + ['EOS'] for text in X_flat]

In [29]:
fastText_gensim.build_vocab(sentences = X_fasttext)

In [30]:
fastText_gensim.train(sentences = X_fasttext, 
                      epochs = 500,
                      total_examples=fastText_gensim.corpus_count)

In [98]:
fastText_gensim.save('Lilly_pkg_modele_linguistique_fastText_bis')
#fastText_gensim = FT_gensim.load('Lilly_pkg_modele_linguistique_fastText_bis')

In [94]:
print(fastText_gensim)

T = fastText_gensim.wv.vectors
print('T est de type {} et de taille {}'.format(type(T), T.shape))

FastText(vocab=7238, size=100, alpha=0.025)
T est de type <class 'numpy.ndarray'> et de taille (7238, 100)


<a id="model_linguistique"></a>

# 1 Modèle Linguistique

Exemples d'implémentation en PyTorch :

- https://github.com/pytorch/examples/blob/master/word_language_model/model.py


Différentes architectures sont décrites dans la litérature :

- Regularizing and Optimizing LSTM Language Models - https://arxiv.org/pdf/1708.02182.pdf

Un modèle linguistique est intérressant en soi, mais peut aussi servir pour le pré-entrainement de couches basses d'un modèle plus complexe :

- Deep contextualized word representations - https://arxiv.org/pdf/1802.05365.pdf
- Improving Language Understanding by Generative Pre-Training - https://www.cs.ubc.ca/~amuham01/LING530/papers/radford2018improving.pdf
- Language Models are Unsupervised Multitask Learners - https://d4mucfpksywv.cloudfront.net/better-language-models/language-models.pdf

### 1.1 Le modèle linguistique

[Retour à la table des matières](#plan)

In [86]:
class LanguageModel(nn.Module):
    def __init__(self, device, 
                 lang,
                 embedding_weights, 
                 hidden_dim,
                 n_layers = 1, 
                 dropout = 0): 
        
        super(LanguageModel, self).__init__()
        # relevant quantities
        self.device = device
        self.lang = lang
        self.hidden_dim = hidden_dim
        self.dropout_p = dropout
        self.n_layers = n_layers               # number of stacked GRU layers
        self.output_dim = hidden_dim           # dimension of outputed rep. of words and utterance
        # parameters
        self.embedding = nn.Embedding(embedding_weights[0], embedding_weights[1]) if type(embedding_weights) == tuple else \
                         nn.Embedding.from_pretrained(torch.FloatTensor(embedding_weights), freeze=True)
        for p in self.embedding.parameters() :
            embedding_dim = p.data.size(1)
        self.dropout = nn.Dropout(p = dropout)
        self.gru = nn.GRU(embedding_dim, 
                          self.hidden_dim, 
                          n_layers,
                          dropout=(0 if n_layers == 1 else dropout), 
                          bidirectional = False)
        self.out = nn.Linear(self.hidden_dim, lang.n_words)
        #self.out.weight = self.embedding.weight
        
        
    # ------------- technical methods ------------------    
    def nbParametres(self) :
        count = 0
        for p in self.parameters():
            if p.requires_grad == True :
                count += p.data.nelement()
        return count
    

    def variableFromSentence(self, sentence):
        '''Turn a sentence into a torch variable, containing a list of indices according
           to a given language.'''
        indexes = [self.lang.word2index[word] for word in sentence if word in self.lang.word2index]          
        result = Variable(torch.LongTensor(indexes).view(-1, 1)).to(self.device)
        return result
    

    def initHidden(self): 
        return Variable(torch.zeros(self.n_layers, 1, self.hidden_dim)).to(self.device)
    
    
    # ------------- main methods ------------------
    def generateWord(self, utterance, hidden = None):
        embeddings = self.embedding(utterance)                          # dim = (input_length, 1, embedding_dim)
        embeddings = self.dropout(embeddings)                           # dim = (input_length, 1, embedding_dim)
        hidden = self.initHidden() if hidden is None else hidden
        _, hidden = self.gru(embeddings, hidden)                        # dim = (input_length, 1, hidden_dim)
        hidden = self.dropout(hidden)
        log_probs = F.log_softmax(self.out(hidden[-1]))         # dim = (1, lang_size)
        return log_probs, hidden  

    
    def forward(self, sentence = ['SOS'], hidden = None, limit = 10, retain = False, color_code = '\033[40m'):
        result = sentence + [color_code]
        count = 0
        stop = False
        utterance = self.variableFromSentence(sentence)
        while not stop :
            log_probs, hidden = self.generateWord(utterance, hidden)
            topv, topi = log_probs[0].data.topk(1)
            ni = topi.item()
            utterance = Variable(torch.LongTensor([[ni]])).to(self.device)
            result.append(self.lang.index2word[ni])
            count += 1
            if count == limit or (type(limit) == str and ni == self.lang.word2index[limit]) or count == 50 :
                stop = True
        if not retain :
            phrase = ' '.join(result)
            phrase = phrase.replace('donc', '=>')
            phrase = re.sub(' \.\s*', '.\n', phrase)
            phrase += '\033[0m'
            print(phrase)
            return
        else :
            return result, hidden   

In [33]:
class LanguageModelTrainer(object):
    def __init__(self, 
                 device,
                 criterion = nn.NLLLoss(reduce = False), #nn.BCEWithLogitsLoss(), #nn.BCELoss(), 
                 optimizer = optim.SGD,
                 print_every=100):
        # relevant quantities
        self.device = device
        self.criterion = criterion
        self.optimizer = optimizer
        self.print_every = print_every
        
        
    def asMinutes(self, s):
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)
    
    
    def timeSince(self, since, percent):
        now = time.time()
        s = now - since
        es = s / (percent)
        rs = es - s
        return '%s (- %s)' % (self.asMinutes(s), self.asMinutes(rs))
        
        
    def trainLoop(self, agent, sentence, word, optimizer, learning_rate):
        """Performs a training loop, with forward pass and backward pass for gradient optimisation."""
        optimizer.zero_grad()
        agent.zero_grad()
        word_var     = agent.variableFromSentence([word]).view(-1)
        sentence_var = agent.variableFromSentence(sentence)
        log_probs, _ = agent.generateWord(sentence_var)
        
        loss = self.criterion(log_probs, word_var)
        topv, topi = log_probs[0].data.topk(1)
        loss_diff = 1 if topi.item() != word_var.data.item() else 0
        loss.backward()
        optimizer.step()                                
        return loss.data[0], loss_diff
        
        
    def train(self,
              agent, 
              sentences,
              n_iters = 100,
              learning_rate=0.01,
              random_state = 42
             ):
        """Performs training over a given dataset and along a specified amount of loops."""
        np.random.seed(random_state)
        start = time.time()
        optimizer = self.optimizer([param for param in agent.parameters() if param.requires_grad == True], lr=learning_rate)
        print_loss_total = 0  
        print_loss_diff_mots_total = 0
        for iter in range(1, n_iters + 1):
            sentence = random.choice(sentences) #--- [str]
            i = random.choice(range(len(sentence)))
            context = ['SOS'] + sentence[:i] 
            target = sentence[i]

            loss, loss_diff_mots = self.trainLoop(agent, context, target, optimizer, learning_rate)
            
            # affichage
            print_loss_total += loss
            print_loss_diff_mots_total += loss_diff_mots       
            if iter % (self.print_every) == 0:
                print_loss_avg = print_loss_total / self.print_every
                print_loss_diff_mots_avg = print_loss_diff_mots_total / self.print_every
                print_loss_total = 0
                print_loss_diff_mots_total = 0
                print('%s (%d %d%%) %.4f %.2f' % (self.timeSince(start, iter / n_iters),
                                             iter, iter / n_iters * 100, 
                                                  print_loss_avg, print_loss_diff_mots_avg))

### 1.2 Entrainement du modèle

In [ ]:
# skipgram custom
#T = skipgram.embedding.weight.cpu().detach().numpy()
#lang = generateLanguageFromNumpy(X, init = 3, lvl = 2)

# fastText gensim
T = fastText_gensim.wv.vectors
lang = generateLanguageFromWordList(list(fastText_gensim.wv.vocab.keys()), init = 0)
lang.addDescriptions(X_flat) # pour retrouver le nombre d'occurences de chaque mot

In [87]:
language_model = LanguageModel(device = device,
                               lang = lang,
                               hidden_dim = 150, #100
                               embedding_weights = T,
                               n_layers = 3, #2
                               dropout = 0.15)
language_model = language_model.to(device)
language_model.nbParametres()

1478138

In [51]:
language_model

LanguageModel(
  (embedding): Embedding(7238, 100)
  (dropout): Dropout(p=0.15)
  (gru): GRU(100, 150, num_layers=3, dropout=0.15)
  (out): Linear(in_features=150, out_features=7238, bias=True)
)

In [38]:
freqs = list(lang.word2count.values())
freqs = [1/np.log(el+1) for el in freqs]
freqs = torch.tensor(freqs).to(device)

In [39]:
print(min(freqs), max(freqs))

tensor(0.0890, device='cuda:0') tensor(1.4427, device='cuda:0')


In [40]:
language_model_trainer = LanguageModelTrainer(device = device, 
                                              criterion = nn.NLLLoss(weight = freqs), 
                                              print_every = 500
                                             )

In [88]:
language_model.load_state_dict(torch.load(path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_bis.pth'))

In [105]:
language_model.train()
#language_model_trainer.train(language_model, X_flat, n_iters = 80000, learning_rate=0.01)
#language_model_trainer.train(language_model, X_flat, n_iters = 40000, learning_rate=0.005)
#language_model_trainer.train(language_model, X_flat, n_iters = 40000, learning_rate=0.002)
language_model_trainer.train(language_model, X_flat, n_iters = 40000, learning_rate=0.001)
#language_model_trainer.train(language_model, X_flat, n_iters = 40000, learning_rate=0.0005)

0m 19s (- 25m 42s) (500 1%) 2.4809 0.42
0m 39s (- 25m 51s) (1000 2%) 2.5120 0.43
0m 54s (- 23m 19s) (1500 3%) 2.4712 0.44
1m 12s (- 22m 58s) (2000 5%) 2.5622 0.44
1m 27s (- 21m 47s) (2500 6%) 2.5426 0.42
1m 40s (- 20m 41s) (3000 7%) 2.4138 0.40
1m 57s (- 20m 29s) (3500 8%) 2.3096 0.40
2m 13s (- 19m 57s) (4000 10%) 2.4725 0.42
2m 28s (- 19m 29s) (4500 11%) 2.5369 0.43
2m 43s (- 19m 5s) (5000 12%) 2.3827 0.40
2m 57s (- 18m 33s) (5500 13%) 2.3754 0.40
3m 12s (- 18m 9s) (6000 15%) 2.4467 0.40
3m 27s (- 17m 49s) (6500 16%) 2.3025 0.40
3m 42s (- 17m 28s) (7000 17%) 2.5415 0.45
3m 56s (- 17m 5s) (7500 18%) 2.4088 0.44
4m 11s (- 16m 47s) (8000 20%) 2.3833 0.41
4m 27s (- 16m 31s) (8500 21%) 2.3885 0.40
4m 44s (- 16m 19s) (9000 22%) 2.2055 0.41
4m 58s (- 15m 59s) (9500 23%) 2.2989 0.40
5m 14s (- 15m 42s) (10000 25%) 2.4319 0.41
5m 29s (- 15m 25s) (10500 26%) 2.4844 0.43
5m 43s (- 15m 6s) (11000 27%) 2.2588 0.40
6m 0s (- 14m 52s) (11500 28%) 2.3465 0.42
6m 14s (- 14m 33s) (12000 30%) 2.6662 0.45


In [106]:
torch.save(language_model.state_dict(), path_to_rep + '\saves\\Lilly_pkg_modele_linguistique_bis.pth')

**Démo**

In [127]:
# fastText gensim, n_layers = 3, dh = 150
language_model.eval()
sentence = random.choice(X_flat)
i = random.choice(range(len(sentence)))
sentence = sentence[:i] if i > 0 else ['SOS']
language_model(sentence, limit = 'EOS', color_code = '\x1b[48;2;255;229;217m' ) #  '\x1b[48;2;255;229;217m' '\x1b[31m'

short description ast pmx pkg apollo lot impossible a demarrer attente apres signature .
description appel de l operateur pour signaler que le limos est en remote pas a signer.
resolve notes.
resume appel de l operateur pour signaler que le limos est en remote pas a signer.
consequence factuelle impossible de poursuivre la checklist.
resolution.
verification de


Recherche d'articles de language model sur les tickets d'incidents dans le contexte de 

- controle qualité
- ouverture de non-conformité
